## Bias scan using Multi-Dimensional Subset Scan (MDSS)

"Identifying Significant Predictive Bias in Classifiers" https://arxiv.org/abs/1611.08292

The goal of bias scan is to identify a subgroup(s) that has significantly more predictive bias than would be expected from an unbiased classifier. There are $\prod_{m=1}^{M}\left(2^{|X_{m}|}-1\right)$ unique subgroups from a dataset with $M$ features, with each feature having $|X_{m}|$ discretized values, where a subgroup is any $M$-dimension
Cartesian set product, between subsets of feature-values from each feature --- excluding the empty set. Bias scan mitigates this computational hurdle by approximately identifing the most statistically biased subgroup in linear time (rather than exponential).


We define the statistical measure of predictive bias function, $score_{bias}(S)$ as a likelihood ratio score and a function of a given subgroup $S$. The null hypothesis is that the given prediction's odds are correct for all subgroups in $\mathcal{D}$:

$$H_{0}:odds(y_{i})=\frac{\hat{p}_{i}}{1-\hat{p}_{i}}\ \forall i\in\mathcal{D}.$$

The alternative hypothesis assumes some constant multiplicative bias in the odds for some given subgroup $S$:

$$H_{1}:\ odds(y_{i})=q\frac{\hat{p}_{i}}{1-\hat{p}_{i}},\ \text{where}\ q>1\ \forall i\in S\ \mathrm{and}\ q=1\ \forall i\notin S.$$

In the classification setting, each observation's likelihood is Bernoulli distributed and assumed independent. This results in the following scoring function for a subgroup $S$:

\begin{align*}
score_{bias}(S)= & \max_{q}\log\prod_{i\in S}\frac{Bernoulli(\frac{q\hat{p}_{i}}{1-\hat{p}_{i}+q\hat{p}_{i}})}{Bernoulli(\hat{p}_{i})}\\
= & \max_{q}\log(q)\sum_{i\in S}y_{i}-\sum_{i\in S}\log(1-\hat{p}_{i}+q\hat{p}_{i}).
\end{align*}
Our bias scan is thus represented as: $S^{*}=FSS(\mathcal{D},\mathcal{E},F_{score})=MDSS(\mathcal{D},\hat{p},score_{bias})$.

where $S^{*}$ is the detected most anomalous subgroup, $FSS$ is one of several subset scan algorithms for different problem settings, $\mathcal{D}$ is a dataset with outcomes $Y$ and discretized features $\mathcal{X}$, $\mathcal{E}$ are a set of expectations or 'normal' values for $Y$, and $F_{score}$ is an expectation-based scoring statistic that measures the amount of anomalousness between subgroup observations and their expectations.

Predictive bias emphasizes comparable predictions for a subgroup and its observations and Bias scan provides a more general method that can detect and characterize such bias, or poor classifier fit, in the larger space of all possible subgroups, without a priori specification.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from aif360.sklearn.datasets import fetch_compas
from aif360.sklearn.metrics import mdss_bias_scan, mdss_bias_score

We'll demonstrate scoring a subset and finding the most anomalous subset with bias scan using the compas dataset.

We can specify subgroups to be scored or scan for the most anomalous subgroup. Bias scan allows us to decide if we aim to identify bias as observing **lower** than predicted probabilities of recidivism, i.e. overestimation, (unprivileged) or observing **higher** than predicted probabilities, i.e. underestimation, (privileged).

Note: categorical features must not be one-hot encoded since scanning those features may find subgroups that are not meaningful e.g., a subgroup with 2 race values.

In [2]:
cols = ['sex', 'race', 'age_cat', 'priors_count', 'c_charge_degree']
X, y = fetch_compas(usecols=cols, binary_race=True)

# Quantize priors count between 0, 1-3, and >3
X['priors_count'] = pd.cut(X['priors_count'], [-1, 0, 3, 100],
                           labels=['0', '1 to 3', 'More than 3'])

### Training
We'll split the dataset and then train a simple classifier to predict the probability of the outcome; (0: Survived, 1: Recidivated)

In [3]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y, train_size=0.7, shuffle=False, random_state=42)

X_train.head()

,,,sex,race,age_cat,priors_count,c_charge_degree
id,sex,race,,,,,
3,Male,African-American,Male,African-American,25 - 45,0,F
4,Male,African-American,Male,African-American,Less than 25,More than 3,F
8,Male,Caucasian,Male,Caucasian,25 - 45,More than 3,F
10,Female,Caucasian,Female,Caucasian,25 - 45,0,M
14,Male,Caucasian,Male,Caucasian,25 - 45,0,F


In [4]:
clf = LogisticRegression(solver='lbfgs', C=1.0, penalty='l2', random_state=0)
clf.fit(X_train.apply(lambda s: s.cat.codes), y_train)
clf.classes_

array(['Recidivated', 'Survived'], dtype=object)

predictions should reflect the probability of a favorable outcome (i.e. no recidivism).

In [5]:
test_prob = clf.predict_proba(X_test.apply(lambda s: s.cat.codes))[:, 1]

In [6]:
df = X_test.copy()
df['observed'] = y_test
df['probabilities'] = test_prob
df.reset_index(drop=True).head()

,sex,race,age_cat,priors_count,c_charge_degree,observed,probabilities
0,Male,African-American,25 - 45,0,F,Survived,0.691005
1,Male,African-American,25 - 45,More than 3,F,Recidivated,0.325263
2,Male,Caucasian,Greater than 45,More than 3,F,Recidivated,0.333202
3,Male,Caucasian,Greater than 45,1 to 3,M,Survived,0.556058
4,Male,African-American,Less than 25,1 to 3,F,Recidivated,0.385969


In this example, we assume that the model makes systematic under or over estimatations of the recidivism risk for certain subgroups and our aim is to identify these subgroups

### Bias scoring

We'll call the bias scoring function and score the test set. The `privileged` argument indicates the direction for which to scan for bias depending on the positive label. In our case since the positive label is 1 ('Survived'), `True` corresponds to checking for underestimated risk of recidivism and `False` corresponds to checking for overestimated risk of recidivism.

In [7]:
print(mdss_bias_score(df['observed'], df['probabilities'], pos_label='Survived',
                      X=df.iloc[:, :-2], subset={'sex': ['Female']},
                      privileged=True))
print(mdss_bias_score(df['observed'], df['probabilities'], pos_label='Survived',
                      X=df.iloc[:, :-2], subset={'sex': ['Male']},
                      privileged=False))

4.8846
0.952


In [8]:
print(mdss_bias_score(df['observed'], df['probabilities'], pos_label='Survived',
                      X=df.iloc[:, :-2], subset={'sex': ['Male']},
                      privileged=True))
print(mdss_bias_score(df['observed'], df['probabilities'], pos_label='Survived',
                      X=df.iloc[:, :-2], subset={'sex': ['Female']},
                      privileged=False))

-0.0
-0.0


If we assume correctly, then our bias score is going to be higher; thus whichever of the assumptions results in a higher bias score has the most evidence of being true. This means females are likely privileged whereas males are likely unpriviledged by our classifier.

### Bias scan
We get the bias score for the apriori defined subgroup but assuming we had no prior knowledge 
about the predictive bias and wanted to find the subgroups with the most bias, we can apply bias scan to identify the priviledged and unpriviledged groups. The privileged argument is not a reference to a group but the direction for which to scan for bias.

In [9]:
privileged_subset = mdss_bias_scan(df['observed'], df['probabilities'],
                                   X=df[df.columns[:-2]], pos_label='Survived',
                                   penalty=0.5, privileged=True)
unprivileged_subset = mdss_bias_scan(df['observed'], df['probabilities'],
                                     X=df[df.columns[:-2]], pos_label='Survived',
                                     penalty=0.5, privileged=False)

Function mdss_bias_scan is deprecated; Change to new interface - aif360.sklearn.detectors.mdss_detector.bias_scan by version 0.5.0.
Function mdss_bias_scan is deprecated; Change to new interface - aif360.sklearn.detectors.mdss_detector.bias_scan by version 0.5.0.


In [10]:
print(privileged_subset)
print(unprivileged_subset)

({'sex': ['Female'], 'age_cat': ['25 - 45', 'Less than 25']}, 9.3413)
({'age_cat': ['Greater than 45']}, 15.1498)


In [11]:
assert privileged_subset[0]
assert unprivileged_subset[0]

We can observe that the bias score is higher than the score of the prior groups. These subgroups are guaranteed to be the highest scoring subgroup among the exponentially many subgroups.

For the purposes of this example, the logistic regression model systematically under estimates the recidivism risk of individuals belonging to the `Female aged less than 25` group. Whereas individuals belonging to the `Greater than 45` age group are assigned a higher risk than is actually observed. We refer to these subgroups as the `detected privileged group` and `detected unprivileged group` respectively.

As noted in the paper, predictive bias is different from predictive fairness so there's no the emphasis in the subgroups having comparable predictions between them. 
We can investigate the difference in what the model predicts vs what we actually observed as well as the multiplicative difference in the odds of the subgroups.

In [12]:
to_choose = df[privileged_subset[0].keys()].isin(privileged_subset[0]).all(axis=1)
temp_df = df.loc[to_choose]

In [13]:
group_obs = temp_df['observed'].cat.codes.mean()
group_prob = 1-temp_df['probabilities'].mean()

"Our detected priviledged group has a size of {}, our model predicts {:.2%} probability of recidivism but we observe {:.2%} as the mean outcome"\
.format(len(temp_df), group_prob, group_obs)

'Our detected priviledged group has a size of 256, our model predicts 34.51% probability of recidivism but we observe 48.05% as the mean outcome'

In [14]:
odds_mul = (group_obs / (1 - group_obs)) / (group_prob /(1 - group_prob))
"This is a multiplicative increase in the odds by {:.3f}".format(odds_mul)

'This is a multiplicative increase in the odds by 1.755'

In [15]:
assert odds_mul > 1

In [16]:
to_choose = df[unprivileged_subset[0].keys()].isin(unprivileged_subset[0]).all(axis=1)
temp_df = df.loc[to_choose]

In [17]:
group_obs = temp_df['observed'].cat.codes.mean()
group_prob = 1-temp_df['probabilities'].mean()

"Our detected unpriviledged group has a size of {}, our model predicts {:.2%} probability of recidivism but we observe {:.2%} as the mean outcome"\
.format(len(temp_df), group_prob, group_obs)

'Our detected unpriviledged group has a size of 309, our model predicts 47.03% probability of recidivism but we observe 32.36% as the mean outcome'

In [18]:
odds_mul = (group_obs / (1 - group_obs)) / (group_prob /(1 - group_prob))
"This is a multiplicative decrease in the odds by {:.3f}".format(odds_mul)

'This is a multiplicative decrease in the odds by 0.539'

In [19]:
assert odds_mul < 1

In summary this notebook demonstrates the use of bias scan to identify subgroups with significant predictive bias, as quantified by a likelihood ratio score, using subset scanning. This allows consideration of not just subgroups of a priori interest or small dimensions, but the space of all possible subgroups of features.
It also presents opportunity for a kind of bias mitigation technique that uses the multiplicative odds in the over-or-under estimated subgroups to adjust for predictive fairness.